TOTAL VALUE OF PRODUCTS SOLD PER MONTH

In [1]:
import numpy as np
import pandas as pd
import json
import re


df = pd.read_json('transactions.json')

filter = df[['transaction_items', 'transaction_value']].copy()

sanitize = df[['transaction_date', 'transaction_items']].copy()
sanitize['transaction_date'] = pd.to_datetime(sanitize.transaction_date).dt.strftime('%m')

df_product_price = filter[(filter['transaction_items'].str.contains('(x1)')) & (~filter['transaction_items'].str.contains(';')) ].drop_duplicates().reset_index(drop=True)
    

month = []
productsPrice = {}
productsTotal = {}


for i, row in df_product_price.iterrows():        
    z = re.search('^(.*?),\(', df_product_price.at[i,'transaction_items']).group(1)     
    if (str(z) not in productsPrice) :
        productsPrice[str(z)] = df_product_price.at[i, 'transaction_value']
    if (str(z) not in productsTotal) :
        productsTotal[str(z)] = [0] * 12 
        
sanitize['transaction_items'] = sanitize['transaction_items'].str.split(';')
sanitize = sanitize.explode('transaction_items').reset_index(drop=True)

for i, row in sanitize.iterrows():
        
    y = re.search('\(x([^)]+)', sanitize.at[i,'transaction_items']).group(1)        
    z = re.search('^(.*?),\(', sanitize.at[i,'transaction_items']).group(1) 
    monthSold = sanitize.at[i,'transaction_date']
    
    productsTotal.get(str(z))[int(monthSold)-1] += (int(y) * int(productsPrice.get(str(z))))
    
parsed_df_sum = pd.DataFrame.from_dict(productsTotal, orient='index')
parsed_df_sum

C:\Users\PB\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,0,1,2,3,4,5,6,7,8,9,10,11
"HealthyKid 3+,Yummy Vegetables",4979500,5128000,4948000,4930500,4867500,4861000,0,0,0,0,0,0
"Candy City,Gummy Worms",1433850,1499400,1497900,1506450,1470150,1490100,0,0,0,0,0,0
"Exotic Extras,Beef Chicharon",12554835,12991299,12750984,12847110,13026372,12862698,0,0,0,0,0,0
"Exotic Extras,Kimchi and Seaweed",7731124,7949251,7963633,7926879,7808627,8073096,0,0,0,0,0,0
"Candy City,Orange Beans",1945026,1997363,1912589,1972886,1982836,2011094,0,0,0,0,0,0
"HealthyKid 3+,Gummy Vitamins",14521500,14970000,15217500,14763000,14922000,14970000,0,0,0,0,0,0
"HealthyKid 3+,Nutrional Milk",19356730,19285090,19653240,19474140,19663190,19436330,0,0,0,0,0,0


NUMBER OF PRODUCTS SOLD PER MONTH

In [2]:
import numpy as np
import pandas as pd
import json
import re
from datetime import datetime

df = pd.read_json('transactions.json')
month = []
products = {}

for i, row in df.iterrows():

    for x in df.at[i,'transaction_items'].split(";"): 
        y = re.search('\(x([^)]+)', x).group(1) 
        z = re.search('^(.*?),\(', x).group(1) 
        
        if (str(z) not in products) :
            products[str(z)] = [0] * 12
        
        
        date = str(df.at[i,'transaction_date']) #get date sold
        dt = datetime.strptime(date, '%Y/%m/%d') #parse date sold and get the month
        #print( int(dt.month) )
    
        products.get(str(z))[int(dt.month)-1] += int(y) 
        
    
parsed_df_productsold = pd.DataFrame.from_dict(products, orient='index')
parsed_df_productsold

,0,1,2,3,4,5,6,7,8,9,10,11
"Exotic Extras,Beef Chicharon",9665,10001,9816,9890,10028,9902,0,0,0,0,0,0
"HealthyKid 3+,Nutrional Milk",9727,9691,9876,9786,9881,9767,0,0,0,0,0,0
"Candy City,Orange Beans",9774,10037,9611,9914,9964,10106,0,0,0,0,0,0
"HealthyKid 3+,Gummy Vitamins",9681,9980,10145,9842,9948,9980,0,0,0,0,0,0
"HealthyKid 3+,Yummy Vegetables",9959,10256,9896,9861,9735,9722,0,0,0,0,0,0
"Candy City,Gummy Worms",9559,9996,9986,10043,9801,9934,0,0,0,0,0,0
"Exotic Extras,Kimchi and Seaweed",9676,9949,9967,9921,9773,10104,0,0,0,0,0,0


CUSTOMER STATUS PER MONTH

In [3]:
sanitize = df[['name', 'transaction_date']].copy()
sanitize['transaction_date'] = pd.to_datetime(sanitize.transaction_date).dt.strftime('%m')
sanitize = sanitize.drop_duplicates()

multi_group = sanitize.groupby(['name'])['transaction_date'].apply(','.join).reset_index()

maxMonth = int(sanitize['transaction_date'].max())

month = []
status = {}

def repeater( monthList ):
    x = 0    
    for month in monthList:
        if ( x != 0 ): 
            if ( int(monthList[x-1]) == int(month)-1 ): 
                status.get('repeaters')[int(month)-1] += 1 
        
        x += 1
        
def inactive( monthList ):
    monthCount = len(monthList)
    y = 0;
    isEarliestMonth = True
    
    for x in range(maxMonth):
        if (x == (int(monthList[y])-1)) :            
            if (y+1 < monthCount):
                y += 1
            if (isEarliestMonth == True): 
                isEarliestMonth = False
        else:
            if (isEarliestMonth == False):
                status.get('inactive')[x] += 1
                
def engaged( monthList ):
    x = 1 
        
    startedOnJanuary = False
    
    for month in monthList: 
        if ( int(month) == 1 ) :
            startedOnJanuary = True
            
        if ( startedOnJanuary and x == int(month) ): 
            x += 1 
        
    
    if ( startedOnJanuary and x == len(monthList) ): 
        for month in monthList:
            status.get('engaged')[int(month)-1] += 1
            
status['repeaters'] = [0] * 12
status['inactive'] = [0] * 12
status['engaged'] = [0] * 12

for i, row in multi_group.iterrows():    
    monthList = row['transaction_date'].split(",") 
    repeater( monthList )
    engaged( monthList )
    inactive( monthList )

parsed_df_customers = pd.DataFrame.from_dict(status, orient='index') 

parsed_df_customers

,0,1,2,3,4,5,6,7,8,9,10,11
repeaters,0,5172,5216,5154,5110,5193,0,0,0,0,0,0
inactive,0,1416,1747,1909,1917,1835,0,0,0,0,0,0
engaged,995,933,798,532,225,682,0,0,0,0,0,0
